In [12]:
import pandas as pd
import numpy as np
import pickle

In [13]:
data = pd.read_csv("data/test.csv")
names = data["Name"]
ids = data["PassengerId"]
data = data.drop(["Name", "PassengerId"], axis = 1)
data["Sex"] = data["Sex"].map({"male": 1, "female": 0})
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})
data = data.drop(["Cabin", "Ticket"], axis = 1)
# data = data.dropna()

In [14]:
data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,2
1,3,0,47.0,1,0,7.0000,0
2,2,1,62.0,0,0,9.6875,2
3,3,1,27.0,0,0,8.6625,0
4,3,0,22.0,1,1,12.2875,0
...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,8.0500,0
414,1,0,39.0,0,0,108.9000,1
415,3,1,38.5,0,0,7.2500,0
416,3,1,NaN,0,0,8.0500,0


# Loading model, predicting and writing predictions

In [21]:
filename = "xgb.sav"
bst = pickle.load(open(filename, "rb"))

In [22]:
preds = bst.predict(data)
df = pd.DataFrame({"PassengerId": ids, "Survived": preds})
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,1
414,1306,1
415,1307,0
416,1308,1


In [28]:
df.to_csv("Submit.csv", index = False)